In [1]:
!pip install google-cloud google-cloud-secret-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.7/116.7 kB 7.5 MB/s eta 0:00:00


In [2]:
from google.cloud import secretmanager
from google.colab import auth


def get_secret(secret_name, proj='aigamer-383501'):
    auth.authenticate_user()
    client = secretmanager.SecretManagerServiceClient()
    response = client.access_secret_version(request={"name": f"projects/{proj}/secrets/{secret_name}/versions/latest"})
    return response.payload.data.decode('UTF-8')


def get_aws_creds():
    aws_key = get_secret('aws-key')
    aws_secret = get_secret('aws-secret')
    return aws_key, aws_secret


def get_github_creds(out_path='github-key'):
    key_str = get_secret('github-key')
    with open(out_path, 'w') as f:
        f.write(key_str)


In [3]:
!git clone https://github.com/yetanotheruseless/labs.git

Cloning into 'labs'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 89 (delta 31), reused 60 (delta 15), pack-reused 0
Unpacking objects: 100% (89/89), 57.96 KiB | 2.07 MiB/s, done.


In [4]:
!cd labs/src

In [5]:
!pwd

/content


In [6]:
aws_key, aws_secret = get_aws_creds()

In [7]:
import sys
sys.path.insert(0, '/content/labs/src')

In [8]:
!cd labs && pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/labs (from -r requirements.txt (line 2))
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.5/227.5 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 94.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.8/57.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 89.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 106.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 5.1 MB/s

In [9]:
from data.make_dataset import *

In [10]:
books_prefix = 'projects/AI-RPG/interim/'
core_rulebook = books_prefix + "The_Strange_Consumer_PDF_Bookmarked_and_Linked_2015-06-23-text.tgz"
out_base = 'data'
!mkdir data


In [11]:
import boto3
import io
import tarfile

def download_and_uncompress_tgzs(aws_key, aws_secret, bucket_name, exclude_string, local_directory):
    # Initialize a boto3 client with the provided AWS credentials
    s3_client = boto3.client(
        's3',
        aws_access_key_id=aws_key,
        aws_secret_access_key=aws_secret
    )

    # List all objects in the S3 bucket
    s3_objects = s3_client.list_objects_v2(Bucket=bucket_name)

    # Iterate through each object and download/uncompress it if it's a .tgz file and doesn't contain the specified string
    for s3_object in s3_objects['Contents']:
        s3_object_key = s3_object['Key']
        if s3_object_key.endswith('.tgz') and exclude_string not in s3_object_key:
            # Download the .tgz file from the S3 bucket into memory
            s3_response = s3_client.get_object(Bucket=bucket_name, Key=s3_object_key)
            tgz_data = s3_response['Body'].read()

            # Load the .tgz data into a file-like buffer
            tgz_buffer = io.BytesIO(tgz_data)

            # Uncompress the .tgz file and extract its contents to the specified local directory
            with tarfile.open(fileobj=tgz_buffer, mode="r:gz") as tar:
                tar.extractall(path=local_directory)

            print(f"Downloaded and uncompressed {s3_object_key} from bucket {bucket_name} to {local_directory}")



In [12]:
import os
import random
import shutil

def create_data_split(source_dir, output_dir, train_ratio=0.8, test_ratio=0.1, valid_ratio=0.1):
    assert train_ratio + test_ratio + valid_ratio == 1, "Ratios must sum to 1."

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for split in ['train', 'test', 'validation']:
        split_dir = os.path.join(output_dir, split)
        if not os.path.exists(split_dir):
            os.makedirs(split_dir)

    all_files = []
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.txt'):
                all_files.append(os.path.join(root, file))

    random.shuffle(all_files)

    train_files = all_files[:int(len(all_files) * train_ratio)]
    test_files = all_files[int(len(all_files) * train_ratio):int(len(all_files) * (train_ratio + test_ratio))]
    valid_files = all_files[int(len(all_files) * (train_ratio + test_ratio)):]

    for file in train_files:
        filename = os.path.basename(file)
        dirname = os.path.basename(os.path.dirname(file))
        new_filename = f"{dirname}_{filename}"
        shutil.copy(file, os.path.join(output_dir, 'train', f"train_{new_filename}"))

    for file in test_files:
        filename = os.path.basename(file)
        dirname = os.path.basename(os.path.dirname(file))
        new_filename = f"{dirname}_{filename}"
        shutil.copy(file, os.path.join(output_dir, 'test', f"test_{new_filename}"))

    for file in valid_files:
        filename = os.path.basename(file)
        dirname = os.path.basename(os.path.dirname(file))
        new_filename = f"{dirname}_{filename}"
        shutil.copy(file, os.path.join(output_dir, 'validation', f"validation_{new_filename}"))


In [13]:
download_and_uncompress_tgzs(aws_key, aws_secret, 'yetanotheruseless-data', "-images", 'data/core')

Downloaded and uncompressed projects/AI-RPG/interim/Alternate_Origins-2015-10-27-text.tgz from bucket yetanotheruseless-data to data/core
Downloaded and uncompressed projects/AI-RPG/interim/Cults_Factions_and_Syndicates_Glimmer-HyperlinkedBookmarked-2016-02-27-text.tgz from bucket yetanotheruseless-data to data/core
Downloaded and uncompressed projects/AI-RPG/interim/Cypher_System_Rulebook-HLBM-2015-07-06-text.tgz from bucket yetanotheruseless-data to data/core
Downloaded and uncompressed projects/AI-RPG/interim/Encyclopedia_of_Impossible_Things-2016-02-04-text.tgz from bucket yetanotheruseless-data to data/core
Downloaded and uncompressed projects/AI-RPG/interim/Impossible_Vehicles-2015-11-16-text.tgz from bucket yetanotheruseless-data to data/core
Downloaded and uncompressed projects/AI-RPG/interim/In_Translation_ST_Character_Options-HLBM-2015-02-23-text.tgz from bucket yetanotheruseless-data to data/core
Downloaded and uncompressed projects/AI-RPG/interim/Mastodon-BMHL-2015-09-19-te

In [14]:
create_data_split('data/core', 'data/training')

In [15]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [16]:
from models.train_model import *


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.9/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('http'), PosixPath('//172.28.0.1'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.9/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc48301118ce

In [17]:
def get_model_and_tokenizers(model_name="facebook/opt-6.7b", torch_dtype=torch.float32):
    os.environ["CUDA_VISIBLE_DEVICES"] = "0"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        load_in_8bit=True,
        device_map='auto',
    )
    model = model.half() if torch_dtype == torch.float16 else model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    for param in model.parameters():
        param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)
    model.gradient_checkpointing_enable()  # reduce number of stored activations
    model.enable_input_require_grads()
    model.lm_head = CastOutputToFloat(model.lm_head)
    config = LoraConfig(
        r=16,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )
    model = get_peft_model(model, config)
    print_trainable_parameters(model)
    return model, tokenizer

In [18]:
model, tokenizer = get_model_and_tokenizers()

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 8388608 || all params: 6666862592 || trainable%: 0.12582542214183376


In [19]:
import transformers
from datasets import load_dataset
data = load_dataset("data/training")
data = data.map(lambda samples: tokenizer(samples['text']), batched=True)

trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        max_steps=500, 
        learning_rate=2e-6, 
        fp16=True,
        tf32=True,
        logging_steps=1, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
) 
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Resolving data files:   0%|          | 0/2320 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/290 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/290 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.9/dist-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,3.973300
2,4.613000
3,5.506400
4,4.248100
5,5.028100
6,4.862700
7,4.400900
8,4.512300
9,4.965400
10,5.080000


TrainOutput(global_step=500, training_loss=4.512078310012817, metrics={'train_runtime': 1523.0495, 'train_samples_per_second': 5.253, 'train_steps_per_second': 0.328, 'total_flos': 1.658470967083008e+16, 'train_loss': 4.512078310012817, 'epoch': 0.1})

In [ ]:
from IPython.display import Markdown as md


batch = tokenizer("What kind of recursions could my character in the game 'The Strange' come from?", return_tensors='pt')
with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, min_length=100, max_new_tokens=400, repetition_penalty=2.0)

md('\n\n' + tokenizer.decode(output_tokens[0], skip_special_tokens=True))